## **Initialize Pinecone**

In [0]:
import pinecone

pinecone.init(api_key="87d21a17-cc6f-4e2d-a4fe-2e5a86ee6fa8", environment="us-west4-gcp-free")

In [0]:
index = pinecone.GRPCIndex("amazon-products")

In [0]:
index.describe_index_stats()

## **Upsert Index**

**Note**: Skip this step if you only want to query the index(semantic search)

In [0]:
import pickle

model_name = "gte_base"
with open(f"embeddings_product_titles_{model_name}.pkl", "rb") as fIn:
    stored_data = pickle.load(fIn)
    asin = stored_data["asin"]
    titles = stored_data["product_titles"]
    embeddings = stored_data["embeddings"]

In [0]:
embeddings = embeddings.tolist()

In [0]:
vectors = list(zip(asin, embeddings))
print("Number of vectors:", len(vectors))

In [0]:
index.upsert(vectors, batch_size=64)

## **Query Index**

Load the dataset in order to later extrart the product title

In [0]:
from pprint import pprint

import delta_sharing

profile_file = "config.share"
client = delta_sharing.SharingClient(profile_file)
print("Available tables:")
pprint(client.list_all_tables())

# load metadata table
table_url = profile_file + "#share__products.silver.amazon_metadata_silver_selected"
df = delta_sharing.load_as_pandas(table_url)

In [0]:
import torch
from sentence_transformers import SentenceTransformer

device = "cuda" if torch.cuda.is_available() else "cpu"

model = SentenceTransformer("thenlper/gte-base", device=device)

In [0]:
query = "microsoft excel"

# create the query vector
xq = model.encode(query).tolist()

# now query
xc = index.query(xq, top_k=5)
for match in xc["matches"]:
    id = match["id"]
    print(df.query("asin == @id")["title"].item())